In [11]:
import time
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import zipfile

import sys
sys.path.append('..')

import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [12]:
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('../jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [13]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

In [14]:
corpus_indices

[990,
 154,
 79,
 682,
 112,
 838,
 420,
 990,
 154,
 440,
 311,
 541,
 445,
 545,
 198,
 8,
 420,
 990,
 154,
 440,
 311,
 437,
 536,
 415,
 859,
 972,
 420,
 437,
 536,
 415,
 545,
 198,
 56,
 420,
 519,
 702,
 101,
 702,
 101,
 702,
 101,
 415,
 990,
 990,
 990,
 990,
 630,
 311,
 420,
 881,
 895,
 521,
 119,
 546,
 420,
 893,
 519,
 253,
 248,
 34,
 866,
 47,
 176,
 348,
 420,
 376,
 834,
 937,
 192,
 428,
 563,
 420,
 893,
 519,
 707,
 445,
 311,
 420,
 413,
 336,
 521,
 893,
 519,
 0,
 303,
 521,
 991,
 930,
 42,
 734,
 420,
 594,
 341,
 521,
 893,
 519,
 192,
 12,
 521,
 991,
 930,
 42,
 734,
 420,
 260,
 942,
 521,
 893,
 519,
 376,
 821,
 521,
 991,
 930,
 42,
 734,
 420,
 201,
 201,
 521,
 893,
 519,
 302,
 410,
 521,
 991,
 930,
 42,
 734,
 420,
 201,
 201,
 521,
 893,
 519,
 302,
 410,
 521,
 991,
 930,
 42,
 734,
 420,
 413,
 336,
 521,
 893,
 519,
 0,
 303,
 521,
 991,
 930,
 42,
 734,
 420,
 594,
 341,
 521,
 893,
 519,
 192,
 12,
 521,
 991,
 930,
 42,
 734,
 420,
 260,

In [15]:
vocab_size

1027

为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为NN（即词典大小vocab_size），每个字符已经同一个从0到N−1N−1的连续整数值索引一一对应。如果一个字符的索引是整数ii, 那么我们创建一个全0的长为NN的向量，并将其位置为ii的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [19]:
def one_hot(x,n_class,dtype = torch.float32):
    #X SHAPE BATCH   OUTPUT SHAPE  BATCH,N-CLASS

    x = x.long()
    res = torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

x = torch.tensor([0, 2])

one_hot(x, vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [20]:
x.shape

torch.Size([2])

In [24]:

x.view(-1,1)


tensor([[0],
        [2]])

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。也就是说，时间步tt的输入为Xt∈Rn×dX
t
​
 ∈R
n×d
 ，其中nn为批量大小，dd为输入个数，即one-hot向量长度（词典大小）。

In [37]:
def to_onehot(X,n_class):
    #X shape:(batch,seq_len) output shape: seq_len elements of (batch,n_class)
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]


In [38]:
X = torch.arange(10).view(2, 5)


X

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [28]:
X.shape

torch.Size([2, 5])

In [29]:
X.shape[1]

5

In [39]:
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1027])


初始化模型参数,接下来，我们初始化模型参数。隐藏单元个数 num_hiddens是一个超参数。

In [41]:
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print('use',device)

use cuda


In [42]:
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)

    #隐藏层参数
    W_xh = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))

    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])






定义模型我们根据循环神经网络的计算表达式实现该模型。首先定义init_rnn_state函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。使用元组是为了更便于处理隐藏状态含有多个NDArray的情况。

In [43]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device))


下面的rnn函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数。3.8节（多层感知机）中介绍过，当元素在实数域上均匀分布时，tanh函数值的均值为0。

In [49]:
def rnn(inputs,state,params):
    #inputs和outputs均为num_steps个形状为（batch_size,vocab_size)的矩阵
    W_xh,W_hh,b_h,W_hq,b_q = params
    H = state
    outputs = []
    for x in inputs:
        H = torch.tanh(torch.matmul(x,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y = torch.matmul(H,W_hq) + b_q
        outputs.append(Y)
    return outputs,H
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([256])


定义预测函数

以下函数基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。这个函数稍显复杂，其中我们将循环神经单元rnn设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [54]:
def pred_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state = init_rnn_state(1,num_hiddens,device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix) - 1):
        #将上一实时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        #计算输出和更新隐藏状态
        (Y,state) = rnn(X,state,params)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

pred_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开颁随爽问师么狗们廓靠'

裁剪梯度循环神经网络中较容易出现梯度衰减或梯度爆炸。我们会在6.6节（通过时间反向传播）中解释原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 gg，并设裁剪的阈值是θθ。裁剪后的梯度

min(θ∥g∥,1)gmin(
∥g∥
θ
​
 ,1)g
的L2L
2
​
 范数不超过θθ。

In [55]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

6.4.6 困惑度
我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下3.4节（softmax回归）中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。
显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小vocab_size。

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

使用困惑度评价模型。
在迭代模型参数前裁剪梯度。
对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考6.3节（语言模型数据集（周杰伦专辑歌词））。
另外，考虑到后面将介绍的其他循环神经网络，为了更通用，这里的函数实现更长一些。

In [98]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()


            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())

            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', pred_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))


num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 70.215869, time 0.12 sec
 - 分开 我想你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
 - 不分开  有有你有你的让我 我有你你 你有我有 你有你的手  哼有你的让我有动的可爱女人 坏坏的让我疯狂的
epoch 100, perplexity 9.689514, time 0.24 sec
 - 分开 我不好好生你 当知不觉 我已经这节奏 后知不觉 你已经这开我 不知不觉 我已经这节奏 后知不觉 你
 - 不分开永 我不能再想你 我知道这样球 你的完美主义 它彻后 我手无 我想就这样牵着你的手不放开 爱能不能够
epoch 150, perplexity 2.923245, time 0.16 sec
 - 分开 有有那里默 的话都著口 不知不觉 你一经离开 白静蜡美 全暖怕日出 白色蜡烛 温暖了空屋 白色蜡烛
 - 不分开吗 我后你 别你我 我想就这样牵着你的手不放开 爱可不能够永远单纯没有悲哀 我 想带你骑单车 我 想
epoch 200, perplexity 1.589947, time 0.32 sec
 - 分开 一使心霜的仪斑 我绪店旁二 三两银够不够 景色入秋 漫天黄沙凉寞 哼哼哈兮 如果我有轻功 飞檐走壁
 - 不分开期 然后将过去 慢慢温习 让我爱上你 那场的剧 是你耿虹 你一定纵我妈大  古前的教育著河的家庭 没
epoch 250, perplexity 1.309939, time 0.09 sec
 - 分开 出在它沙去的豆瓣酱 我 伤物 静战螂 一直怎么三步四步望著天 看星星 一颗两颗三颗四颗 连成线一著
 - 不分开扫 我叫你爸 你打我妈 这样对吗干嘛这样 何必让它牵鼻子走 瞎 说狼三木斯中出进难现 泥板上的字迹依


In [99]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

RuntimeError: Can't detach views in-place. Use detach() instead. If you are using DistributedDataParallel (DDP) for training, and gradient_as_bucket_view is set as True, gradients are views of DDP buckets, and hence detach_() cannot be called on these gradients. To fix this error, please refer to the Optimizer.zero_grad() function in torch/optim/optimizer.py as the solution.